# Read the Source Words Dataset

In [1]:
import json # Extracting data from the .json files
from collections import defaultdict # Making higher dimensional dictionaries

def source_verb(dataset_file):
    
    # parsing the json file and storing it in an object
    with open(dataset_file, 'r', encoding = 'utf-8') as f:
        source = json.load(f)
        
    # Setting all the words in the corpus in a more accessible dictionary,
    # where dictionary maps the tag of the language to the words
    source_word = defaultdict(list)
    for x in source:
        for y in x:
            words = x[y].split()
            for word in words:
                source_word[y].append(word)
        
    return source_word

## Phase 1: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb

In [2]:
def infinitive_verb_search(source_verb, target_verb, input_verb, infinitive_verb_dataset):
    
    translation = []
    
    for i in range(len(input_verb)):
        for j in infinitive_verb_dataset.get(source_verb):
            if input_verb[i] == j:
                # Get the index of the source word and use it to get it's equivalent in target word
                translation.append(infinitive_verb_dataset.get(target_verb)[infinitive_verb_dataset.get(source_verb).index(j)])

    if len(translation) == 0:
        translation.append("")
     
    return translation

## Phase 2: If the input word isn't an infinitive verb, check what kind of verb is it. (Verb tense recognition)
### Translation (Turkish to Arabic)

In [3]:
def tr_to_ar_translation(input_verb, infinitive_verb_dataset):
    
    # Phase 1: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
    translation = infinitive_verb_search('tr', 'ar', input_verb, infinitive_verb_dataset)

    # Phase 2: If the input verb isn't an infinitive verb, check what kind of verb is it. (Verb tense recognition)
    if translation[0] == "":
        
        # Future tense (Gelecek zaman)
        if input_verb[0][-4:] == "acak" or input_verb[0][-4:] == "ecek" or input_verb[0][-4:] == "acağ" or input_verb[0][-4:] == "eceğ" or input_verb[0][-6:-2] == "acak" or input_verb[0][-6:-2] == "ecek" or input_verb[0][-6:-2] == "acağ" or input_verb[0][-6:-2] == "eceğ" or input_verb[0][-7:-3] == "acak" or input_verb[0][-7:-3] == "ecek" or input_verb[0][-7:-3] == "acağ" or input_verb[0][-7:-3] == "eceğ":
            
            # Phase 3: Stemming the Turkish verb from the Future Tense and the Pronoun letters
            tr_verb_stem = tr_verb_stemming(input_verb, "FT")
            
            # Phase 4: Convert the Turkish Stem verb into an Infinitive verb
            tr_stem_to_infinitive = tr_verb_stem_to_infinitive(tr_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            tr_infinitive_verb_search = infinitive_verb_search('tr', 'ar', tr_stem_to_infinitive, infinitive_verb_dataset)
            
            if tr_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Arabic infinitive verb to the correct verb tense and pronoun
                translation = ar_verb_infinitive_to_tense(tr_infinitive_verb_search, tr_verb_stem[0], "FT")
            
            
        # Present Continuous tense (Şimdiki zaman)
        elif input_verb[0][-3:] == "yor" or input_verb[0][-5:-2] == "yor" or input_verb[0][-6:-3] == "yor":
            
            # Phase 3: Stemming the Turkish verb from the Present Continuous Tense and the Pronoun letters
            tr_verb_stem = tr_verb_stemming(input_verb, "PCT")
            
            # Phase 4: Convert the Turkish Stem verb into an Infinitive verb
            tr_stem_to_infinitive = tr_verb_stem_to_infinitive(tr_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            tr_infinitive_verb_search = infinitive_verb_search('tr', 'ar', tr_stem_to_infinitive, infinitive_verb_dataset)
            
            if tr_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Arabic infinitive verb to the correct verb tense and pronoun
                translation = ar_verb_infinitive_to_tense(tr_infinitive_verb_search, tr_verb_stem[0], "PCT")
            
            
        # Past tense (Geçmiş zaman)
        elif input_verb[0][-2:] == "dı" or input_verb[0][-2:] == "di" or input_verb[0][-2:] == "du" or input_verb[0][-2:] == "dü" or input_verb[0][-3:-1] == "dı" or input_verb[0][-3:-1] == "di" or input_verb[0][-3:-1] == "du" or input_verb[0][-3:-1] == "dü" or input_verb[0][-2:] == "tı" or input_verb[0][-2:] == "ti" or input_verb[0][-2:] == "tu" or input_verb[0][-2:] == "tü" or input_verb[0][-3:-1] == "tı" or input_verb[0][-3:-1] == "ti" or input_verb[0][-3:-1] == "tu" or input_verb[0][-3:-1] == "tü":
            
            # Phase 3: Stemming the Turkish verb from the Past Tense and the Pronoun letters
            tr_verb_stem = tr_verb_stemming(input_verb, "PT")
            
            # Phase 4: Convert the Turkish Stem verb into an Infinitive verb
            tr_stem_to_infinitive = tr_verb_stem_to_infinitive(tr_verb_stem[1])
            
            # Phase 5: Search for the input verb in the infinitive verb dataset, if its exists, get the Target verb
            tr_infinitive_verb_search = infinitive_verb_search('tr', 'ar', tr_stem_to_infinitive, infinitive_verb_dataset)
            
            if tr_infinitive_verb_search[0] == "":
                print("Sorry, the verb doesn't exist in our database!")
                
            else:
                # Phase 6: Convert the Turkish infinitive verb to the correct verb tense and pronoun
                translation = ar_verb_infinitive_to_tense(tr_infinitive_verb_search, tr_verb_stem[0], "PT")

            
    return translation

# Turkish to Arabic

## Phase 3: Verb stemming. Remove the letters related to the tenses (Future, Present, or Past) and the pronoun

In [4]:
def tr_verb_stemming(input_verb, verb_tense):
    
    # Future Tense
    if verb_tense == "FT":
        verb_variation = []
        
        # Step 1: Check the Turkish suffix personal pronoun and delete them
        if input_verb[0][-2:] == "ım" or input_verb[0][-2:] == "im": # Check last 2 letters
            pronoun = "1PS" # 1st Person Singular Pronoun "Ben"
            verb_variation.append(input_verb[0][:-2]) # Append the verb to the list without the suffix personal pronoun (last 2 letters)
            
        elif input_verb[0][-3:] == "sın" or input_verb[0][-3:] == "sin": # Check last 3 letters
            pronoun = "2PS" # 2nd Person Singular Pronoun "Sen"
            verb_variation.append(input_verb[0][:-3]) # Append the verb to the list without the suffix personal pronoun (last 3 letters)
            
        else:
            pronoun = "3PS" # 3rd Person Singular Pronoun "O"
            verb_variation.append(input_verb[0]) # Append the verb to the list
            
        
        # Step 2: Delete the Turkish Future Tense suffix "acak", "ecek", "acağ", OR "eceğ" (last 4 letters)
        if verb_variation[0][-4:] == "acak" or verb_variation[0][-4:] == "ecek" or verb_variation[0][-4:] == "acağ" or verb_variation[0][-4:] == "eceğ": # Check last 4 letters
            verb_variation[0] = verb_variation[0][:-4]
            
            
        # Step 3: If the verb ends with "y", delete it
        if verb_variation[0][-1:] == "y":
            verb_variation[0] = verb_variation[0][:-1]
            
            

    # Present Continuous Tense
    elif verb_tense == "PCT":
        verb_variation = []

        # Step 1: Check the Turkish suffix personal pronoun and delete them
        if input_verb[0][-2:] == "um": # Check last 2 letters
            pronoun = "1PS" # 1st Person Singular Pronoun "Ben"
            verb_variation.append(input_verb[0][:-2]) # Append the verb to the list without the suffix personal pronoun (last 2 letters)
            
        elif input_verb[0][-3:] == "sun": # Check last 3 letters
            pronoun = "2PS" # 2nd Person Singular Pronoun "Sen"
            verb_variation.append(input_verb[0][:-3]) # Append the verb to the list without the suffix personal pronoun (last 3 letters)
            
        else:
            pronoun = "3PS" # 3rd Person Singular Pronoun "O"
            verb_variation.append(input_verb[0]) # Append the verb to the list
            
            
        # Step 2: Delete the Turkish Present Continuous Tense suffix "yor" (last 3 letters)
        if verb_variation[0][-3:] == "yor": # Check last 3 letters
            verb_variation[0] = verb_variation[0][:-3]
            
            
        # Step 3: Since there is no specific rule to know which vowel the verb ends with or without, 
        # we get the all variations the verb ends with or without a vowel
        
        # List holding the turkish vowel letters and an empty string
        vowels = ["a", "ı", "e", "i", "o", "u", "ö", "ü"]
        
        
        # First, delete the vowel letter (last letter)
        verb_variation[0] = verb_variation[0][:-1]
        
        
        # Second, create a string containing the verb
        str_verb_variation = verb_variation[0]
        
        
        # Third, get all the variations of the verb with or without vowel letters
        for i in range(8):
            verb_variations = str_verb_variation + vowels[i]
            verb_variation.append(verb_variations)
            
            
    # Past Tense
    elif verb_tense == "PT":
        verb_variation = []
        
        # Step 1: Check the Turkish suffix personal pronoun and delete them
        # Condition 1: 1st Person Singular Pronoun "Ben"|"أنا"
        if input_verb[0][-1:] == "m": # Check last letter
            pronoun = "1PS" # 1st Person Singular Pronoun "Ben"
            verb_variation.append(input_verb[0][:-1]) # Append the verb to the list without the suffix personal pronoun (last letter)
            
        elif input_verb[0][-1:] == "n": # Check last letter
            pronoun = "2PS" # 2nd Person Singular Pronoun "Sen"
            verb_variation.append(input_verb[0][:-1]) # Append the verb to the list without the suffix personal pronoun (last letter)
            
        else:
            pronoun = "3PS" # 3rd Person Singular Pronoun "O"
            verb_variation.append(input_verb[0]) # Append the verb to the list
            
        
        # Step 2: Delete the Turkish Past Tense suffix "dı", "di", "du", or "dü" (last 2 letters)
        if verb_variation[0][-2:] == "dı" or verb_variation[0][-2:] == "di" or verb_variation[0][-2:] == "du" or verb_variation[0][-2:] == "dü" or verb_variation[0][-2:] == "tı" or verb_variation[0][-2:] == "ti" or verb_variation[0][-2:] == "tu" or verb_variation[0][-2:] == "tü": # Check last 2 letters
            verb_variation[0] = verb_variation[0][:-2]

            
    tr_verb_stem = [pronoun, verb_variation]
    
    return tr_verb_stem

## Phase 4: Convert the Turkish Stem verb into an Infinitive verb

In [5]:
def tr_verb_stem_to_infinitive(stem_verb):
    
    infinitive_verb = stem_verb
    
    for i in range(len(infinitive_verb)):
        # Condition 1: If verb last vowel is "a", "ı", "o", or "u"
        if infinitive_verb[i][-1:] == "a" or infinitive_verb[i][-1:] == "ı" or infinitive_verb[i][-1:] == "o" or infinitive_verb[i][-1:] == "u" or infinitive_verb[i][-2:-1] == "a" or infinitive_verb[i][-2:-1] == "ı" or infinitive_verb[i][-2:-1] == "o" or infinitive_verb[i][-2:-1] == "u":
            infinitive_verb[i] = infinitive_verb[i] + "mak" # Add the suffix "mak" to the end of the stem verb
            
    
        # Condition 2: If verb last vowel is "e", "i", "ö", or "ü"
        if infinitive_verb[i][-1:] == "e" or infinitive_verb[i][-1:] == "i" or infinitive_verb[i][-1:] == "ö" or infinitive_verb[i][-1:] == "ü" or infinitive_verb[i][-2:-1] == "e" or infinitive_verb[i][-2:-1] == "i" or infinitive_verb[i][-2:-1] == "ö" or infinitive_verb[i][-2:-1] == "ü":
            infinitive_verb[i] = infinitive_verb[i] + "mek" # Add the suffix "mak" to the end of the stem verb
    
    return infinitive_verb

## Phase 6: Convert the Arabic infinitive verb to the correct verb tense and pronoun

In [6]:
def ar_verb_infinitive_to_tense(infinitive_verb, pronoun, verb_tense):

    # Each string in the list tokenize it into characters
    for i in range(len(infinitive_verb)):
        infinitive_verb[i] = list(infinitive_verb[i])
    
    # In case of Future Tense and Present Continuous Tense
    if verb_tense == "FT" or verb_tense == "PCT":
        
        for i in range(len(infinitive_verb)):
        
            # Step 1: Check the infinitive letter
            # Condition 1: If the word starts with "إ"
            if infinitive_verb[i][0] == "إ":
                del infinitive_verb[i][0] # delete the first letter "إ"

                # Special conditions
                if infinitive_verb[i][-1] == "ء" and infinitive_verb[i][-2] == "ا": # If last 2 letters are "اء"
                    del infinitive_verb[i][-1] # Delete "ء" as the last letter
                    del infinitive_verb[i][-1] # Delete "ا" as the last letter
                    infinitive_verb[i].append("ى") # Append "ى" as the last letter

                elif infinitive_verb[i][-1] == "ة" and infinitive_verb[i][-3] == "ا": # If last letter is "ة" and before the last letter in 2 is "ا"
                    del infinitive_verb[i][-1] # Delete "ة" as the last letter
                    infinitive_verb[i][-2] = "ي" # Replace "ا" with "ي"

                if infinitive_verb[i][-2] == "ا": # If there is a letter "ا" before the last letter, delete it
                    del infinitive_verb[i][-2]


            # Condition 2: If second letter has "شدة"
            elif infinitive_verb[i][2] == "ّ":

                del infinitive_verb[i][0] # delete the first letter "ت"

                # Special conditions
                # If last letter 2 letters are "ية", turns them into "ى"
                if infinitive_verb[i][-1] == "ة" and infinitive_verb[i][-2] == "ي":
                    del infinitive_verb[i][-1] # Delete "ة" as the last letter
                    del infinitive_verb[i][-1] # Delete "ي" as the last letter
                    infinitive_verb[i].append("ى") # Append "ى" as the last letter

                # If the letter before the last is "ي", delete it
                if infinitive_verb[i][-2] == "ي":
                    del infinitive_verb[i][-2] # Delete "ي" as the letter before the last


            # Condition 3: If second letter is "ا"
            elif infinitive_verb[i][2] == "ا":
                del infinitive_verb[i][0] # Delete the first letter "م"
                del infinitive_verb[i][-1] # Delete the last letter "ة"


            # Condition 4: If first letter is "ت"
            elif infinitive_verb[i][0] == "ت":
                
                # Special conditions
                if infinitive_verb[i][-1] == "ي": # If the last letter is "ي", replace it with "ى"
                    infinitive_verb[i][-1] = "ى"
                else:
                    infinitive_verb[i] = infinitive_verb[i] # Keep the verb as it is


            # Condition 5: If non of the above conditions is correct
            else:
                del infinitive_verb[i][-1] # Delete the last letter "ة"



            # Step 2: Check the correct verb tense of the source verb
            # Future Tense
            if verb_tense == "FT":

                # Step 3: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "أنا"|"Ben"
                if pronoun == "1PS":
                    # Step 4: Add the pronoun prefix
                    infinitive_verb[i].insert(0, "أ") # Insert "أ"

                    # Step 5: Add the Future Tense prefix "س"
                    infinitive_verb[i].insert(0, "س")

                    # Step 6: Make the splited string into one string
                    infinitive_verb[i] = "".join(infinitive_verb[i]) 


                # Condition 2: 2nd Person Singular Pronoun "أنتَ" OR "أنتِ"|"Sen"
                elif pronoun == "2PS":
                    # Step 4: Add the pronoun prefix
                    infinitive_verb[i].insert(0, "ت") # Insert "ت"

                    # Step 5: Add the Future Tense prefix "س"
                    infinitive_verb[i].insert(0, "س")

                    infinitive_verb_2PMS = infinitive_verb[i] # 2nd Person Male Singular Pronoun "أنتَ"

                    # Step 6: Make the splited string into one string
                    join_infinitive_verb_2PMS = "".join(infinitive_verb_2PMS)
                    
                    infinitive_verb_2PFS = infinitive_verb[i]
                    
                    if infinitive_verb_2PFS[-1] == "ي":
                        infinitive_verb_2PFS.append("ن") # 2nd Person Female Singular Pronoun "أنتِ"
                    else:
                        infinitive_verb_2PFS.append("ي") # 2nd Person Female Singular Pronoun "أنتِ"
                        infinitive_verb_2PFS.append("ن")

                    # Step 6: Make the splited string into one string
                    join_infinitive_verb_2PFS = "".join(infinitive_verb_2PFS) 

                    infinitive_verb[i] = join_infinitive_verb_2PMS + " | " + join_infinitive_verb_2PFS


                # Condition 3: 3rd Person Singular Pronoun "هي" OR "هو"|O"
                elif pronoun == "3PS":
                    
                    # 3rd Person Male Singular Pronoun "هو"
                    # Step 4: Add the pronoun prefix
                    infinitive_verb_3PMS = infinitive_verb[i]
                    
                    infinitive_verb_3PMS.insert(0, "ي") # Insert "ي"

                    # Step 5: Add the Future Tense prefix "س"
                    infinitive_verb_3PMS.insert(0, "س")

                    # Step 6: Make the splited string into one string
                    join_infinitive_verb_3PMS = "".join(infinitive_verb_3PMS)

 
                    # 3rd Person Female Singular Pronoun "هي"
                    # Step 4: Add the pronoun prefix
                    infinitive_verb_3PFS = infinitive_verb[i]
                    
                    infinitive_verb_3PFS[1] = "ت" # Replacing second letter with "ت"

                    # Step 5: Add the Future Tense prefix "س"
                    infinitive_verb_3PFS[0] = "س"

                    # Step 6: Make the splited string into one string
                    join_infinitive_verb_3PFS = "".join(infinitive_verb_3PFS)

                    infinitive_verb[i] = join_infinitive_verb_3PMS + " | " + join_infinitive_verb_3PFS

 

            # Present Continuous Tense
            if verb_tense == "PCT":

                # Step 3: Check the correct pronoun of the source verb
                # Condition 1: 1st Person Singular Pronoun "أنا"|"Ben"
                if pronoun == "1PS":
                    # Step 4: Add the pronoun prefix AND the Present Continuous Tense prefix
                    infinitive_verb[i].insert(0, "أ") # Insert "أ"

                    # Step 5: Make the splited string into one string
                    infinitive_verb[i] = "".join(infinitive_verb[i]) 

                    
                # Condition 2: 2nd Person Singular Pronoun "أنتَ" OR "أنتِ"|"Sen"
                elif pronoun == "2PS":
                    # Step 4: Add the pronoun prefix AND the Present Continuous Tense prefix
                    infinitive_verb[i].insert(0, "ت") # Insert "ت"

                    infinitive_verb_2PMS = infinitive_verb[i] # 2nd Person Male Singular Pronoun "أنتَ"

                    # Step 5: Make the splited string into one string
                    join_infinitive_verb_2PMS = "".join(infinitive_verb_2PMS)
                    
                    infinitive_verb_2PFS = infinitive_verb[i]

                    if infinitive_verb_2PFS[-1] == "ي":
                        infinitive_verb_2PFS.append("ن") # 2nd Person Female Singular Pronoun "أنتِ"
                    else:
                        infinitive_verb_2PFS.append("ي") # 2nd Person Female Singular Pronoun "أنتِ"
                        infinitive_verb_2PFS.append("ن")

                    # Step 5: Make the splited string into one string
                    join_infinitive_verb_2PFS = "".join(infinitive_verb_2PFS) 

                    infinitive_verb[i] = join_infinitive_verb_2PMS + " | " + join_infinitive_verb_2PFS
                    
                    
                # Condition 3: 3rd Person Singular Pronoun "هي" OR "هو"|O"
                elif pronoun == "3PS":

                    # 3rd Person Male Singular Pronoun "هو"
                    # Step 4: Add the pronoun prefix AND the Present Continuous Tense prefix
                    infinitive_verb_3PMS = infinitive_verb[i]
                    
                    infinitive_verb_3PMS.insert(0, "ي") # Insert "ي"

                    # Step 5: Make the splited string into one string
                    join_infinitive_verb_3PMS = "".join(infinitive_verb_3PMS)

                    # 3rd Person Female Singular Pronoun "هي"
                    # Step 4: Add the pronoun prefix AND the Present Continuous Tense prefix
                    infinitive_verb_3PFS = infinitive_verb[i]
                    
                    infinitive_verb_3PFS[0] = "ت" # Replacing first letter with "ت"

                    # Step 5: Make the splited string into one string
                    join_infinitive_verb_3PFS = "".join(infinitive_verb_3PFS)

                    infinitive_verb[i] = join_infinitive_verb_3PMS + " | " + join_infinitive_verb_3PFS
        
    

    # Past Tense
    if verb_tense == "PT":
        
        for i in range(len(infinitive_verb)):
        
            # Step 1: Check the infinitive letter
            # Condition 1: If the word starts with "إ"
            if infinitive_verb[i][0] == "إ":
                
                if len(infinitive_verb[i]) < 5: # In case the length of the verb is less than 5 change "إ" to "أ", otherwise leave it as it is
                    infinitive_verb[i][0] = "أ" # Replace the first letter "إ" with "أ"
                    

                # Special conditions
                if infinitive_verb[i][-1] == "ء" and infinitive_verb[i][-2] == "ا": # If last 2 letters are "اء"
                    del infinitive_verb[i][-1] # Delete "ء" as the last letter
                    del infinitive_verb[i][-1] # Delete "ا" as the last letter
                    infinitive_verb[i].append("ي") # Append "ي" as the last letter

                elif infinitive_verb[i][-1] == "ة" and infinitive_verb[i][-3] == "ا": # If last letter is "ة" and before the last letter in 2 is "ا"
                    del infinitive_verb[i][-1] # Delete "ة" as the last letter
                    infinitive_verb[i][-2] = "ي" # Replace "ا" with "ي"

                if infinitive_verb[i][-2] == "ا": # If there is a letter "ا" before the last letter, delete it
                    del infinitive_verb[i][-2]


            # Condition 2: If second letter has "شدة"
            elif infinitive_verb[i][2] == "ّ":
                
                del infinitive_verb[i][0] # delete the first letter "ت"

                # Special conditions
                # If last letter 2 letters are "ية", turns them into "ى"
                if infinitive_verb[i][-1] == "ة" and infinitive_verb[i][-2] == "ي":
                    del infinitive_verb[i][-1] # Delete "ة" as the last letter
                    del infinitive_verb[i][-1] # Delete "ي" as the last letter
                    infinitive_verb[i].append("ى") # Append "ى" as the last letter

                # If the letter before the last is "ي", delete it
                if infinitive_verb[i][-2] == "ي":
                    del infinitive_verb[i][-2] # Delete "ي" as the letter before the last
        

            # Condition 3: If second letter is "ا"
            elif infinitive_verb[i][2] == "ا":
                del infinitive_verb[i][0] # Delete the first letter "م"
                del infinitive_verb[i][-1] # Delete the last letter "ة"


            # Condition 4: If first letter is "ت"
            elif infinitive_verb[i][0] == "ت":
                if infinitive_verb[i][-1] == "ي": # If the last letter is "ي", replace it with "ى"
                    infinitive_verb[i][-1] = "ى"
                else:
                    infinitive_verb[i] = infinitive_verb[i] # Keep the verb as it is


            # Condition 5: If non of the above conditions is correct
            else:
                del infinitive_verb[i][-1] # Delete the last letter "ة"
            
        
            # Step 3: Check the correct pronoun of the source verb
            # Condition 1: 1st Person Singular Pronoun "أنا"|"Ben"
            # OR 2nd Person Singular Pronoun "أنتَ" OR "أنتِ"|"Sen"
            if pronoun == "1PS" or pronoun == "2PS":
                # Step 4: Add the pronoun prefix
                if infinitive_verb[i][-1] == "ة":
                    infinitive_verb[i][-1] = "ت"
                else:
                    infinitive_verb[i].append("ت") # Append "ت"

                # Step 5: Make the splited string into one string
                infinitive_verb[i] = "".join(infinitive_verb[i]) 


            # Condition 2: 3rd Person Singular Pronoun "هي" OR "هو"|O"
            elif pronoun == "3PS":

                # 3rd Person Male Singular Pronoun "هو"
                # Step 3: Add the pronoun prefix
                infinitive_verb_3PMS = infinitive_verb[i]
                
                if infinitive_verb_3PMS[-1] == "ة": # If the letter "ة" exists at the end of the verb delete it
                    del infinitive_verb_3PMS[-1]
                    
                if infinitive_verb_3PMS[-1] == "ي": # If the letter "ي" exists at the end of the verb change it to "ى"
                    infinitive_verb_3PMS[-1] = "ى"

                # Step 5: Make the splited string into one string
                join_infinitive_verb_3PMS = "".join(infinitive_verb_3PMS)


                # 3rd Person Female Singular Pronoun "هي"
                # Step 3: Add the pronoun prefix
                infinitive_verb_3PFS = infinitive_verb[i]
                
                if infinitive_verb_3PFS[-1] == "ة":
                    infinitive_verb_3PFS[-1] = "ت"
                else:
                    infinitive_verb_3PFS[-1] = infinitive_verb_3PFS[-1] + "ت" # Append "ت"

                # Step 5: Make the splited string into one string
                join_infinitive_verb_3PFS = "".join(infinitive_verb_3PFS)

                infinitive_verb[i] = join_infinitive_verb_3PMS + " | " + join_infinitive_verb_3PFS

                
    return infinitive_verb

In [7]:
def main():

    dataset_file = 'data.json' #data file with the translations
    
    source_verb_dataset = source_verb(dataset_file) # Read the dataset and tag each word with its language
    
    # Translation Program
    print("Welcome to the Arabic/Turkish Translator!")
    print("Would you like to translate from: \n 1) Arabic to Turkish \n 2) Turkish to Arabic")
    translate_choice = input()
    
    if translate_choice == "1":
        print("Enter the word:")
        print("AR:")
        input_word = [input()] # Placing the verb written by the user in a list
        word_search = ar_to_tr_translation(input_word, source_word_dataset) # Predict the verb!
        print("TR:")
        print(verb_search)
        
    elif translate_choice == "2":
        print("Enter the word:")
        print("TR:")
        input_verb = [input()] # Placing the verb written by the user in a list
        verb_search = tr_to_ar_translation(input_verb, source_verb_dataset) # Predict the verb!
        print("AR:")
        print(verb_search)

main()

Welcome to the Arabic/Turkish Translator!
Would you like to translate from: 
 1) Arabic to Turkish 
 2) Turkish to Arabic
2
Enter the word:
TR:
izledi
AR:
['شاهد | شاهدت']
